# Build the Neural Network

> [original article](https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html)

> original code is somewhat modified in my taste

Neural networks comprise of layers/modules that perform operations on data. The `torch.nn` namespace provides all the building blocks you need to build your own NN (neural network). Every module in PyTorch subclasses the `nn.Module`. A neural network is a module itself that consists of other modules (layers). This nested structure allows for building and managing complex architectures easily.

In the following sections, we'll build a neural network to classify images in the FashionMNIST dataset.

In [9]:
import os
import torch as th
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## Get Device for Training

We want to be able to train our model on hardware accelerator like the GPU, if it is available. Let's check to see if `torch.cuda` is available, else we continue to use CPU.

In [10]:
device = "cuda" if th.cuda.is_available() else "cpu"
print(f"Using {device} device.")

Using cuda device.


## Define the Class

We define our neural network by subclassing `nn.Module`, and initialize the neural network layers in `__init__`. Every `nn.Module` subclass implements the operations on input data in the `forward` method.

In [11]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

We create an instance of `NeuaralNetwork`, and move it ot the `device`, and print its structure.

In [12]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the model, we pass it the input data. This executes the model's `forward`, along with some **background operations**. Do not call `model.forward()` directly!

Calling the model on the input returns a 10-dimensional tensor with raw predicted values for each class. We get the prediction probabilities by passing it trough an instance of the `nn.Softmax` module

In [13]:
X = th.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([5], device='cuda:0')


## Model layers

Breakdown of layers in FashionMNIST model. To illustrate it, we will take a sample minibatch of 3 images of size 28x28 and see what happens to it as we pass it trough the network.

In [14]:
input_image = th.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


### `nn.Flatten`
We initialize the `nn.Flatten` layer to convert 2D 28x28 image into contigous array of 784 pixel values (the minibatch dimension at `dim=0` is maintained).


In [15]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### `nn.Linear`
The linear layer is a module that applies linear transformation on the input using its stored weights and biases.


In [16]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### nn.ReLU

Non-linear activations are what create complex mappinsg between the model's inputs and outputs. They are applied after linear transformations to introduce *nonlinearity*, helping neural networks learn a wide variety of phenomena.

In this model we use `nn.ReLu` between our linear layers, but there's other activations to introduce non-linearity in your model.

In [17]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 4.1243e-01,  1.9555e-01,  1.0197e-01, -9.6868e-03,  2.2294e-01,
         -3.5581e-01,  5.1306e-02,  1.0318e-01, -5.0564e-01,  1.9930e-01,
         -2.4768e-01, -4.8948e-01,  1.9197e-01,  2.0246e-01,  4.4489e-01,
          1.3658e-01, -4.0475e-01,  8.1602e-02,  3.8424e-01, -3.6548e-01],
        [ 5.7087e-01, -1.1311e-01, -2.3483e-02,  2.9339e-01, -1.1987e-01,
          1.9318e-01, -2.5548e-01, -1.4805e-02, -4.5089e-02,  3.5981e-01,
         -3.8972e-01, -7.5222e-01,  1.9089e-01, -2.0054e-01,  2.2500e-01,
          8.7991e-02, -1.8426e-01,  1.4401e-01, -3.4560e-02, -5.4551e-01],
        [ 1.7723e-01, -1.2920e-01, -2.5830e-01,  7.4404e-02, -1.8009e-02,
         -1.8871e-01, -3.5387e-05, -8.5037e-02, -1.1079e-02,  1.2330e-01,
         -5.0635e-01, -5.3148e-01, -6.1477e-03,  1.9012e-01,  4.1513e-01,
          1.3919e-01, -4.3690e-01,  1.4088e-01,  4.7113e-01, -2.0844e-01]],
       grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.4124, 0.1956, 0.1020, 0.0000, 0.2229,

### nn.Sequential
`nn.Sequential` is an ordered container of modules. The data is passed trough all the modules the same order as defined. You can use sequential containers to put together a quick network lik `seq_modules`.


In [19]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = th.rand(3,28,28)
logits = seq_modules(input_image)

### `nn.Softmax `
The last linear layer of the neural network returns logits - raw values in `[-infty, infty]` - which are passed to `nn.Softmax` module. The logits are scaled to values `[0, 1]` representing the model's predicted probabilities for each class, `dim` parameter indicates the dimension along which the values must sum to 1.

In [ ]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)